In [ ]:
import os
import pandas as pd

In [ ]:
import vineyard
vineyard.init()

In [ ]:
### Fetch raw dataset
import vineyard
vineyard_store = vineyard.connect(os.environ['VINEYARD_IPC_SOCKET'])

In [ ]:
# read using vineyard's io driver
dataset = vineyard.io.read(
    table='feast-oss.fraud_tutorial.transactions',
    columns=['src_account', 'timestamp', 'is_fraud'])

dataset.head()

In [ ]:
vineyard_store.put('fraud_tutorial_transactions', dataset)

In [ ]:
### Data preprocessing
import vineyard
vineyard_store = vineyard.connect(os.environ['VINEYARD_IPC_SOCKET'])

In [ ]:
from datetime import datetime, timedelta

now = datetime.now()
two_days_ago = datetime.now() - timedelta(days=2)

dataset = vineyard_store.get('fraud_tutorial_transactions', dataset)

# filter
dataset = dataset[dataset['timestamp'] >= two_days_ago.isoformat() & dataset['timestamp'] <= now]

def apply_feature_rules(dataset):
    training_data = dataset
    training_data['user_transaction_count_7d'] = compute_7d_count(dataset)
    training_data['credit_score'] = join_credit_score(dataset)
    return training_data

# apply feature view rules
training_data = apply_feature_rules(dataset)

training_data.head()

In [ ]:
vineyard_store.put('fraud_tutorial_transactions_training_data', training_data)

In [ ]:
### Model training
import vineyard
vineyard_store = vineyard.connect(os.environ['VINEYARD_IPC_SOCKET'])

In [ ]:
from sklearn.linear_model import LinearRegression

training_data = vineyard_store.get('fraud_tutorial_transactions_training_data')

# Drop stray nulls
training_data.dropna(inplace=True)

# Select training matrices
X = training_data[[
    "transaction_count_7d", 
    "credit_score",
    'timestamp',
]]
y = training_data["is_fraud"]

# Train a simple SVC model
model = LinearRegression()
model.fit(X, y)

In [ ]:
# Saving the model to vineyard
vineyard_store.put('fraud_tutorial_transactions_sklearn_model', model)

In [ ]:
#### Infererence using trained model
import vineyard
vineyard_store = vineyard.connect(os.environ['VINEYARD_IPC_SOCKET'])

In [ ]:
# Obtaining the model from vineyard
model = vineyard_store.get('fraud_tutorial_transactions_sklearn_model')

# Get first two rows of training data
dataset = vineyard.io.read(
    table='feast-oss.fraud_tutorial.transactions_unknown',
    columns=['src_account', 'timestamp', 'is_fraud'])
dataset = apply_feature_rules(dataset)

# Select a sample
samples = dataset.iloc[:2]

# Make a test prediction
model.predict(samples)

In [ ]:
#### Infererence using trained model and save results
import vineyard

In [ ]:
# Make prediction for all
prediction = model.predict(dataset)

# write using vineyard's io driver
vineyard.io.write(
    prediction,
    table='feast-oss.fraud_tutorial.transactions_prediction',
    columns=['src_account', 'is_fraud'])